In [1]:
import findspark
findspark.init('/usr/local/spark/')
import pyspark
from pyspark.sql import SparkSession
from pyspark import SparkContext


In [2]:
print(pyspark.__version__)

3.3.1


In [3]:
mongo_ip = "mongodb://localhost:27017/Yelp_Database."

In [4]:
from pyspark import SparkContext
# spark = SparkSession.builder.appName("Trail Session").getOrCreate()
spark = SparkSession \
    .builder \
    .appName("myApp") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .getOrCreate()

23/03/11 01:33:09 WARN Utils: Your hostname, paritosh-GF63-Thin-9SCXR resolves to a loopback address: 127.0.1.1; using 192.168.83.74 instead (on interface enp3s0)
23/03/11 01:33:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/usr/local/spark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/paritosh/.ivy2/cache
The jars for the packages stored in: /home/paritosh/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-2ddcdede-e65d-400d-8845-b32f4c54346a;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 191ms :: artifacts dl 10ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artif

23/03/11 01:33:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


### Loading data from Mongo DB to Saprk

In [5]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "yelp_business").load()

df.show(5)

23/03/11 01:33:22 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|                 _id|             address|          attributes|         business_id|          categories|         city|               hours|is_open|  latitude|   longitude|                name|postal_code|review_count|stars|state|
+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+--------------------+-------+----------+------------+--------------------+-----------+------------+-----+-----+
|{63f3abfceaecb55d...|1616 Chapala St, ...|{null, null, null...|Pns2l4eNsfO8kk83d...|Doctors, Traditio...|Santa Barbara|    

In [6]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- AgesAllowed: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |    |-- 

In [45]:
df_temp = df.drop('_id', 'attributes', 'address', 'hours', 'postal_code','latitude','longitude')
# creating new dataframe by dropin few columns from old datframe

### Schema Details
- business_id: string, 22 character unique string business id
- name: string, the business's name
- address: string, the city
- state: string, 2 character state code, if applicable
- city: string, the city
- postal_code: string, the postal code
- latitude: float, latitude
- longitude: float, longitude
- stars: float, star rating, rounded to half-stars
- review_count: integer, number of reviews
- is_open: integer, 0 or 1 for closed or open, respectively
- categories: array of strings of business categories
- attributes: object, business attributes to values. note: some attribute values might be objects

In [8]:
df[["business_id", "name", "categories", ]].show()

+--------------------+--------------------+--------------------+
|         business_id|                name|          categories|
+--------------------+--------------------+--------------------+
|Pns2l4eNsfO8kk83d...|Abby Rappoport, L...|Doctors, Traditio...|
|mpf3x-BjTdTEA3yCZ...|       The UPS Store|Shipping Centers,...|
|tUFrWirKiKi_TAnsV...|              Target|Department Stores...|
|MTSW4McQd7CbVtyjq...|  St Honore Pastries|Restaurants, Food...|
|mWMc6_wTdE0EUBKIG...|Perkiomen Valley ...|Brewpubs, Breweri...|
|CF33F8-E6oudUQ46H...|      Sonic Drive-In|Burgers, Fast Foo...|
|n_0UpQx1hsNbnPUSl...|     Famous Footwear|Sporting Goods, F...|
|qkRM_2X51Yqxk3btl...|      Temple Beth-El|Synagogues, Relig...|
|k0hlBqXX-Bt0vf1op...|Tsevi's Pub And G...|Pubs, Restaurants...|
|bBDDEgkFA1Otx9Lfe...|      Sonic Drive-In|Ice Cream & Froze...|
|UJsufbvfyfONHeWdv...|           Marshalls|Department Stores...|
|eEOYSgkmpB90uNA7l...|Vietnamese Food T...|Vietnamese, Food,...|
|il_Ro8jwPlHresjw9...|   

### Using Temporary views 

In [9]:
df.createOrReplaceTempView("yelp_business")

In [10]:
trial1 = spark.sql("select business_id, name, stars from yelp_business")

In [11]:
trial1.show()

+--------------------+--------------------+-----+
|         business_id|                name|stars|
+--------------------+--------------------+-----+
|Pns2l4eNsfO8kk83d...|Abby Rappoport, L...|  5.0|
|mpf3x-BjTdTEA3yCZ...|       The UPS Store|  3.0|
|tUFrWirKiKi_TAnsV...|              Target|  3.5|
|MTSW4McQd7CbVtyjq...|  St Honore Pastries|  4.0|
|mWMc6_wTdE0EUBKIG...|Perkiomen Valley ...|  4.5|
|CF33F8-E6oudUQ46H...|      Sonic Drive-In|  2.0|
|n_0UpQx1hsNbnPUSl...|     Famous Footwear|  2.5|
|qkRM_2X51Yqxk3btl...|      Temple Beth-El|  3.5|
|k0hlBqXX-Bt0vf1op...|Tsevi's Pub And G...|  3.0|
|bBDDEgkFA1Otx9Lfe...|      Sonic Drive-In|  1.5|
|UJsufbvfyfONHeWdv...|           Marshalls|  3.5|
|eEOYSgkmpB90uNA7l...|Vietnamese Food T...|  4.0|
|il_Ro8jwPlHresjw9...|             Denny's|  2.5|
|jaxMSoInw8Poo3XeM...|        Adams Dental|  5.0|
|0bPLkL0QhhPO5kt1_...|Zio's Italian Market|  4.5|
|MUTTqe8uqyMdBl186...|            Tuna Bar|  4.0|
|rBmpy_Y1UbBx8ggHl...|Arizona Truck Out...|  4.5|


In [12]:
df[['state']].distinct().count()
# there are 50 states in the US, data for only 27 states available

27

In [13]:
spark.sql("SELECT * , CASE WHEN attributes.DogsAllowed='True' THEN 'Yes' ELSE 'No' END AS DogsAllowed, \
                      CASE WHEN attributes.RestaurantsTakeOut='True' THEN 'Yes' ELSE 'No' END AS RestaurantsTakeOut, \
                      CASE WHEN attributes.HappyHour='True' THEN 'Yes' ELSE 'No' END AS HappyHour, \
                      CASE WHEN attributes.OutdoorSeating='True' THEN 'Yes' ELSE 'No' END AS OutdoorSeating, \
                      CASE WHEN attributes.GoodForKids='True' THEN 'Yes' ELSE 'No' END AS GoodForKids, \
                      CASE WHEN attributes.HasTV='True' THEN 'Yes' ELSE 'No' END AS HasTV, \
                      CASE WHEN attributes.RestaurantsReservations='True' THEN 'Yes' ELSE 'No' END As RestaurantsReservations, \
                      CASE WHEN attributes.WheelchairAccessible='True' THEN 'Yes' ELSE 'No' END As WheelchairAccessible from yelp_business").createOrReplaceTempView("Biz")
#spark.sql("select * from Biz").show() 

In [14]:
spark.sql("SELECT * ,CASE WHEN attributes.RestaurantsPriceRange2 LIKE '1' THEN 1 \
                     WHEN attributes.RestaurantsPriceRange2 LIKE '2' THEN 2 \
                     WHEN attributes.RestaurantsPriceRange2 LIKE '3' THEN 3 \
                     WHEN attributes.RestaurantsPriceRange2 LIKE '4' THEN 4 \
                     ELSE 'None' \
                    End AS RestaurantsPriceRange from Biz ").createOrReplaceTempView("Biz")

In [15]:
spark.sql("SELECT * ,CASE WHEN attributes.WiFi LIKE '%paid%' THEN 'paid' \
                     WHEN attributes.WiFi LIKE '%free%' THEN 'free' \
                     ELSE 'No' \
                    End AS Wifi from Biz ").createOrReplaceTempView("Biz")

In [16]:
spark.sql("SELECT * ,CASE WHEN attributes.NoiseLevel LIKE '%average%' THEN 'average' \
                     WHEN attributes.NoiseLevel LIKE '%very_loud%' THEN 'very_loud' \
                     WHEN attributes.NoiseLevel LIKE '%loud%' THEN 'loud' \
                     WHEN attributes.NoiseLevel LIKE '%quiet%' THEN 'quiet' \
                     ELSE 'No' \
                     End AS Noiselevel from Biz ").createOrReplaceTempView("Biz")
#Very loud before loud or very l

In [17]:
spark.sql("SELECT * ,CASE WHEN attributes.Alcohol LIKE '%full_bar%' THEN 'full_bar' \
                     WHEN attributes.Alcohol LIKE '%beer_and_wine%' THEN 'beer_and_wine' \
                     ELSE 
          'No' \
                    End AS Alcohol from Biz ").createOrReplaceTempView("Biz")

In [18]:
spark.sql("SELECT * ,CASE WHEN attributes.DriveThru='True' THEN 'True' \
                     ELSE 'False' \
                    End AS DriveThru from Biz ").createOrReplaceTempView("Biz")

In [19]:
business1=spark.sql("select * ,\
          CASE WHEN categories like '%Restaurants%' THEN 'Yes' \
          ELSE 'No' End AS is_restaurants\
          From Biz")
          

business1.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- address: string (nullable = true)
 |-- attributes: struct (nullable = true)
 |    |-- AcceptsInsurance: string (nullable = true)
 |    |-- Alcohol: string (nullable = true)
 |    |-- Ambience: string (nullable = true)
 |    |-- BYOB: string (nullable = true)
 |    |-- BYOBCorkage: string (nullable = true)
 |    |-- BestNights: string (nullable = true)
 |    |-- BikeParking: string (nullable = true)
 |    |-- BusinessAcceptsBitcoin: string (nullable = true)
 |    |-- BusinessAcceptsCreditCards: string (nullable = true)
 |    |-- BusinessParking: string (nullable = true)
 |    |-- ByAppointmentOnly: string (nullable = true)
 |    |-- Caters: string (nullable = true)
 |    |-- CoatCheck: string (nullable = true)
 |    |-- Corkage: string (nullable = true)
 |    |-- DietaryRestrictions: string (nullable = true)
 |    |-- DogsAllowed: string (nullable = true)
 |    |-- DriveThru: string (nullable = true)
 |

In [21]:
business1=business1.drop("_id")
business1=business1.drop("attributes")
business1=business1.drop("hours")
business1.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Al

In [29]:
business1.createOrReplaceTempView("biz")

In [30]:
spark.sql("select NoiseLevel from biz where NoiseLevel = 'very_loud'").count()

1200

In [31]:
spark.sql("select attributes.NoiseLevel from yelp_business where attributes.NoiseLevel like '%very_loud%'").count()

1200

In [32]:
business1.createOrReplaceTempView("business")

## Saving the Extracted Dataset in CSV format on local PC

In [77]:
business1.coalesce(1).write.option("header",True).csv("/home/dbda/Project/dataset/business_cleaned")

Change paths accordingly

In [96]:
business1.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongo_ip + "business_cleaned").save()

***Getting counts of Unique Categories (Don't Run unless count is needed)***

In [68]:
cat = spark.sql("select categories from business")

In [18]:
rdd1 = cat.rdd
# converting dataframe(name-cat) to rdd(name-rdd1)

In [19]:
rdd1.collect()
#rdd operation .collect to show the data

[Row(categories='Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists'),
 Row(categories='Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services'),
 Row(categories='Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores'),
 Row(categories='Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries'),
 Row(categories='Brewpubs, Breweries, Food'),
 Row(categories='Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants'),
 Row(categories='Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories'),
 Row(categories='Synagogues, Religious Organizations'),
 Row(categories='Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek'),
 Row(categories='Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food'),
 Row(categories='Department Stores, Shopping, Fashion'),
 Row(categories='Vietnamese, Food, Restaurants, Food Trucks'),
 Row(

In [20]:
rdd2 = rdd1.flatMap(lambda x : x)
rdd2.collect()
# The flatMap transformation is similar to the map transformation, but it can produce zero or more output elements 
# for each input element. The flatMap method takes a function that maps each input element to an iterable 
# (e.g. list, tuple, etc.) of output elements. Then, it flattens the resulting iterables into a single RDD.

# In this case, the lambda function lambda x: x simply returns each input element as is. 
# So, the flatMap transformation will flatten the RDD rdd1 into a single RDD rdd2 containing all 
# the elements of rdd1 in a flat structure.

# Note that if the elements of rdd1 are themselves iterable (e.g. if rdd1 contains lists or tuples), 
# then the flatMap transformation would flatten the RDD at two levels, i.e. it would produce a single 
# flattened list of all the elements in all the lists or tuples.

['Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services',
 'Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores',
 'Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries',
 'Brewpubs, Breweries, Food',
 'Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants',
 'Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories',
 'Synagogues, Religious Organizations',
 'Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek',
 'Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food',
 'Department Stores, Shopping, Fashion',
 'Vietnamese, Food, Restaurants, Food Trucks',
 'American (Traditional), Restaurants, Diners, Breakfast & Brunch',
 'General Dentistry, Dentists, Health & Medical, Cosmetic Dentists',
 'Food, Delis, Italian, Bakeries, Restaurants',
 'Sushi Bars

In [21]:
rdd3 = rdd2.map(lambda x : str(x))
rdd3.collect()
# Note that this transformation is not necessary if all the elements of rdd2 are already strings,
# as str() would have no effect on them.

['Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services',
 'Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores',
 'Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries',
 'Brewpubs, Breweries, Food',
 'Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants',
 'Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories',
 'Synagogues, Religious Organizations',
 'Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek',
 'Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food',
 'Department Stores, Shopping, Fashion',
 'Vietnamese, Food, Restaurants, Food Trucks',
 'American (Traditional), Restaurants, Diners, Breakfast & Brunch',
 'General Dentistry, Dentists, Health & Medical, Cosmetic Dentists',
 'Food, Delis, Italian, Bakeries, Restaurants',
 'Sushi Bars

In [22]:
rdd4 = rdd3.flatMap(lambda x : x.split(','))
lister = rdd4.collect()

In [23]:
setter = set(lister)

In [24]:
len(setter)

2455

In [25]:
lister2= list(setter)

In [26]:
lister2=[x.strip() for x in lister2]
lister2

['Talent Agencies',
 'Home Theatre Installation',
 'Video/Film Production',
 'Hong Kong Style Cafe',
 'Water Purification Services',
 'Attraction Farms',
 'Home Decor',
 'Paint & Sip',
 'Honduran',
 'Newspapers & Magazines',
 'Carpet Cleaning',
 'Hardware Stores',
 'Astrologers',
 'Self Storage',
 'Comfort Food',
 'Gemstones & Minerals',
 'Christmas Trees',
 'General Dentistry',
 'Interior Design',
 'Guns & Ammo',
 'Wedding Chapels',
 'Fabric Stores',
 'Mortgage Brokers',
 'Dim Sum',
 'Eyewear & Opticians',
 'Food Stands',
 'Japanese Curry',
 'Teeth Whitening',
 'Irish',
 'Dinner Theater',
 'Dance Schools',
 'Blow Dry/Out Services',
 'Opera & Ballet',
 'Breweries',
 'Brasseries',
 'Active Life',
 'Tanning Beds',
 'Public Transportation',
 'Preschools',
 'Bike Repair/Maintenance',
 'Lawn Services',
 'Emergency Pet Hospital',
 'Cooking Schools',
 'Lice Services',
 'Proctologists',
 'Motorsport Vehicle Dealers',
 'Cabinetry',
 'Team Building Activities',
 'Real Estate Law',
 'Laundry Serv

In [27]:
setter2= set(lister2)

In [28]:
len(setter2)

1312

In [29]:
list2 = rdd3.collect()
list2

['Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists',
 'Shipping Centers, Local Services, Notaries, Mailbox Centers, Printing Services',
 'Department Stores, Shopping, Fashion, Home & Garden, Electronics, Furniture Stores',
 'Restaurants, Food, Bubble Tea, Coffee & Tea, Bakeries',
 'Brewpubs, Breweries, Food',
 'Burgers, Fast Food, Sandwiches, Food, Ice Cream & Frozen Yogurt, Restaurants',
 'Sporting Goods, Fashion, Shoe Stores, Shopping, Sports Wear, Accessories',
 'Synagogues, Religious Organizations',
 'Pubs, Restaurants, Italian, Bars, American (Traditional), Nightlife, Greek',
 'Ice Cream & Frozen Yogurt, Fast Food, Burgers, Restaurants, Food',
 'Department Stores, Shopping, Fashion',
 'Vietnamese, Food, Restaurants, Food Trucks',
 'American (Traditional), Restaurants, Diners, Breakfast & Brunch',
 'General Dentistry, Dentists, Health & Medical, Cosmetic Dentists',
 'Food, Delis, Italian, Bakeries, Restaurants',
 'Sushi Bars

In [30]:
rdd5 = rdd3.map(lambda x : x.split(','))
rdd5.collect()

[['Doctors',
  ' Traditional Chinese Medicine',
  ' Naturopathic/Holistic',
  ' Acupuncture',
  ' Health & Medical',
  ' Nutritionists'],
 ['Shipping Centers',
  ' Local Services',
  ' Notaries',
  ' Mailbox Centers',
  ' Printing Services'],
 ['Department Stores',
  ' Shopping',
  ' Fashion',
  ' Home & Garden',
  ' Electronics',
  ' Furniture Stores'],
 ['Restaurants', ' Food', ' Bubble Tea', ' Coffee & Tea', ' Bakeries'],
 ['Brewpubs', ' Breweries', ' Food'],
 ['Burgers',
  ' Fast Food',
  ' Sandwiches',
  ' Food',
  ' Ice Cream & Frozen Yogurt',
  ' Restaurants'],
 ['Sporting Goods',
  ' Fashion',
  ' Shoe Stores',
  ' Shopping',
  ' Sports Wear',
  ' Accessories'],
 ['Synagogues', ' Religious Organizations'],
 ['Pubs',
  ' Restaurants',
  ' Italian',
  ' Bars',
  ' American (Traditional)',
  ' Nightlife',
  ' Greek'],
 ['Ice Cream & Frozen Yogurt',
  ' Fast Food',
  ' Burgers',
  ' Restaurants',
  ' Food'],
 ['Department Stores', ' Shopping', ' Fashion'],
 ['Vietnamese', ' Food', 

In [31]:
rdd6 = rdd5.map(lambda x : [y.strip() for y in x])
rdd6.collect()

[['Doctors',
  'Traditional Chinese Medicine',
  'Naturopathic/Holistic',
  'Acupuncture',
  'Health & Medical',
  'Nutritionists'],
 ['Shipping Centers',
  'Local Services',
  'Notaries',
  'Mailbox Centers',
  'Printing Services'],
 ['Department Stores',
  'Shopping',
  'Fashion',
  'Home & Garden',
  'Electronics',
  'Furniture Stores'],
 ['Restaurants', 'Food', 'Bubble Tea', 'Coffee & Tea', 'Bakeries'],
 ['Brewpubs', 'Breweries', 'Food'],
 ['Burgers',
  'Fast Food',
  'Sandwiches',
  'Food',
  'Ice Cream & Frozen Yogurt',
  'Restaurants'],
 ['Sporting Goods',
  'Fashion',
  'Shoe Stores',
  'Shopping',
  'Sports Wear',
  'Accessories'],
 ['Synagogues', 'Religious Organizations'],
 ['Pubs',
  'Restaurants',
  'Italian',
  'Bars',
  'American (Traditional)',
  'Nightlife',
  'Greek'],
 ['Ice Cream & Frozen Yogurt', 'Fast Food', 'Burgers', 'Restaurants', 'Food'],
 ['Department Stores', 'Shopping', 'Fashion'],
 ['Vietnamese', 'Food', 'Restaurants', 'Food Trucks'],
 ['American (Traditio

In [32]:
lister3 = rdd6.collect()

In [33]:
dict1={}
{dict1.update({y:0}) for y in list(setter2)}

{None}

In [34]:
lister3

[['Doctors',
  'Traditional Chinese Medicine',
  'Naturopathic/Holistic',
  'Acupuncture',
  'Health & Medical',
  'Nutritionists'],
 ['Shipping Centers',
  'Local Services',
  'Notaries',
  'Mailbox Centers',
  'Printing Services'],
 ['Department Stores',
  'Shopping',
  'Fashion',
  'Home & Garden',
  'Electronics',
  'Furniture Stores'],
 ['Restaurants', 'Food', 'Bubble Tea', 'Coffee & Tea', 'Bakeries'],
 ['Brewpubs', 'Breweries', 'Food'],
 ['Burgers',
  'Fast Food',
  'Sandwiches',
  'Food',
  'Ice Cream & Frozen Yogurt',
  'Restaurants'],
 ['Sporting Goods',
  'Fashion',
  'Shoe Stores',
  'Shopping',
  'Sports Wear',
  'Accessories'],
 ['Synagogues', 'Religious Organizations'],
 ['Pubs',
  'Restaurants',
  'Italian',
  'Bars',
  'American (Traditional)',
  'Nightlife',
  'Greek'],
 ['Ice Cream & Frozen Yogurt', 'Fast Food', 'Burgers', 'Restaurants', 'Food'],
 ['Department Stores', 'Shopping', 'Fashion'],
 ['Vietnamese', 'Food', 'Restaurants', 'Food Trucks'],
 ['American (Traditio

In [35]:
dict1

{'Home Theatre Installation': 0,
 'Attraction Farms': 0,
 'Portable Toilet Services': 0,
 'Paint & Sip': 0,
 'Newspapers & Magazines': 0,
 'Carpet Cleaning': 0,
 'Hardware Stores': 0,
 'Self Storage': 0,
 'Comfort Food': 0,
 'Gemstones & Minerals': 0,
 'Interior Design': 0,
 'Dim Sum': 0,
 'Eyewear & Opticians': 0,
 'Food Stands': 0,
 'Stonemasons': 0,
 'Teeth Whitening': 0,
 'Irish': 0,
 'Dance Schools': 0,
 'Blow Dry/Out Services': 0,
 'Brasseries': 0,
 'Tanning Beds': 0,
 'Bike Repair/Maintenance': 0,
 'Preschools': 0,
 'Lawn Services': 0,
 'Cooking Schools': 0,
 'Real Estate Law': 0,
 'Laundry Services': 0,
 'Costumes': 0,
 'Officiants': 0,
 'Mexican': 0,
 'Motorsport Vehicle Repairs': 0,
 'Walk-in Clinics': 0,
 'Pet Waste Removal': 0,
 'Fitness/Exercise Equipment': 0,
 'Elder Care Planning': 0,
 'Workers Compensation Law': 0,
 'Refinishing Services': 0,
 'Halal': 0,
 'Waxing': 0,
 'Pet Groomers': 0,
 'Bookkeepers': 0,
 'Keys & Locksmiths': 0,
 'Marketing': 0,
 'Education': 0,
 'Tr

In [36]:
setter2

{'Home Theatre Installation',
 'Attraction Farms',
 'Portable Toilet Services',
 'Paint & Sip',
 'Newspapers & Magazines',
 'Carpet Cleaning',
 'Hardware Stores',
 'Self Storage',
 'Comfort Food',
 'Gemstones & Minerals',
 'Interior Design',
 'Dim Sum',
 'Eyewear & Opticians',
 'Food Stands',
 'Stonemasons',
 'Teeth Whitening',
 'Irish',
 'Dance Schools',
 'Blow Dry/Out Services',
 'Brasseries',
 'Tanning Beds',
 'Bike Repair/Maintenance',
 'Preschools',
 'Lawn Services',
 'Cooking Schools',
 'Real Estate Law',
 'Laundry Services',
 'Costumes',
 'Officiants',
 'Mexican',
 'Motorsport Vehicle Repairs',
 'Walk-in Clinics',
 'Pet Waste Removal',
 'Fitness/Exercise Equipment',
 'Elder Care Planning',
 'Workers Compensation Law',
 'Refinishing Services',
 'Halal',
 'Waxing',
 'Pet Groomers',
 'Bookkeepers',
 'Keys & Locksmiths',
 'Marketing',
 'Education',
 'Traditional Chinese Medicine',
 'Jewelry Repair',
 'Flowers',
 'Food Tours',
 'Traditional Clothing',
 'Bike Shop',
 'Dog Parks',
 'Ev

In [37]:
[{dict1.update({y:(dict1[y]+1)}) if y in x else 0 for x in lister3} for y in list(setter2)]

[{0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0, None},
 {0,

In [38]:
dict1

{'Home Theatre Installation': 138,
 'Attraction Farms': 25,
 'Portable Toilet Services': 4,
 'Paint & Sip': 109,
 'Newspapers & Magazines': 42,
 'Carpet Cleaning': 526,
 'Hardware Stores': 792,
 'Self Storage': 795,
 'Comfort Food': 746,
 'Gemstones & Minerals': 31,
 'Interior Design': 509,
 'Dim Sum': 140,
 'Eyewear & Opticians': 1016,
 'Food Stands': 423,
 'Stonemasons': 1,
 'Teeth Whitening': 122,
 'Irish': 215,
 'Dance Schools': 102,
 'Blow Dry/Out Services': 456,
 'Brasseries': 47,
 'Tanning Beds': 130,
 'Bike Repair/Maintenance': 273,
 'Preschools': 344,
 'Lawn Services': 156,
 'Cooking Schools': 76,
 'Real Estate Law': 47,
 'Laundry Services': 1149,
 'Costumes': 137,
 'Officiants': 44,
 'Mexican': 4600,
 'Motorsport Vehicle Repairs': 19,
 'Walk-in Clinics': 148,
 'Pet Waste Removal': 10,
 'Fitness/Exercise Equipment': 98,
 'Elder Care Planning': 7,
 'Workers Compensation Law': 35,
 'Refinishing Services': 148,
 'Halal': 362,
 'Waxing': 2333,
 'Pet Groomers': 1549,
 'Bookkeepers'

In [39]:
dict2 = sorted(dict1.items(), key = lambda x : -x[1])
dict2

[('Restaurants', 52268),
 ('Food', 27781),
 ('Shopping', 24395),
 ('Home Services', 14356),
 ('Beauty & Spas', 14292),
 ('Nightlife', 12281),
 ('Health & Medical', 11890),
 ('Local Services', 11198),
 ('Bars', 11065),
 ('Automotive', 10773),
 ('Event Planning & Services', 9895),
 ('Sandwiches', 8366),
 ('American (Traditional)', 8139),
 ('Active Life', 7687),
 ('Pizza', 7093),
 ('Coffee & Tea', 6703),
 ('Fast Food', 6472),
 ('Breakfast & Brunch', 6239),
 ('American (New)', 6097),
 ('Hotels & Travel', 5857),
 ('Home & Garden', 5799),
 ('Fashion', 5739),
 ('Burgers', 5636),
 ('Arts & Entertainment', 5434),
 ('Auto Repair', 5433),
 ('Hair Salons', 5046),
 ('Nail Salons', 4621),
 ('Mexican', 4600),
 ('Italian', 4573),
 ('Specialty Food', 4233),
 ('Doctors', 3763),
 ('Pets', 3758),
 ('Real Estate', 3577),
 ('Seafood', 3539),
 ('Fitness & Instruction', 3293),
 ('Professional Services', 3270),
 ('Hair Removal', 3239),
 ('Desserts', 3186),
 ('Chinese', 3169),
 ('Bakeries', 3150),
 ('Grocery', 

In [40]:
import pandas as pd
dataf = pd.DataFrame(dict2,columns=['Categories','Frequency'])
dataf

,Categories,Frequency
0,Restaurants,52268
1,Food,27781
2,Shopping,24395
3,Home Services,14356
4,Beauty & Spas,14292
...,...,...
1307,DUI Schools,1
1308,Pathologists,1
1309,Water Suppliers,1
1310,Faith-based Crisis Pregnancy Centers,1


In [41]:
dataf.head(50)

,Categories,Frequency
0,Restaurants,52268
1,Food,27781
2,Shopping,24395
3,Home Services,14356
4,Beauty & Spas,14292
5,Nightlife,12281
6,Health & Medical,11890
7,Local Services,11198
8,Bars,11065
9,Automotive,10773


In [47]:
dataf.to_csv("Categorycount.csv",index=False)

## Isolating the Restaurants and then selecting certain categories from them ##

In [92]:
res = spark.sql("select * from business where categories like '%Restaurants%'")

In [93]:
res.createOrReplaceTempView("restaurants")

In [94]:
res.count()

52268

In [95]:
res.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Al

In [96]:
res = res.drop('is_restaurants')

In [97]:
res.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Al

In [98]:
res = res.filter(res.state == "FL")
# creating new dataframe using filter which contains only florida data
# filter() method in PySpark is used to extract a subset of rows from a DataFrame that satisfy a given Boolean condition.

In [99]:
res.count()

8731

In [41]:
reviewdf = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "yelp_review").load()
# reads data from a MongoDB database collection named 'yelp_review' and creates a DataFrame named reviewdf

In [43]:
reviewdf = reviewdf.drop("_id")

In [44]:
reviewdf.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- user_id: string (nullable = true)



In [50]:
reviewdf = reviewdf.drop("cool", "date", "funny", "text", "useful")

In [51]:
reviewdf.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- user_id: string (nullable = true)



In [48]:
reviewdf.count()

6990280

In [100]:
res = res.withColumnRenamed("stars","restaurant_stars").withColumnRenamed("business_id","bus_business_id")

In [101]:
res.printSchema()

root
 |-- address: string (nullable = true)
 |-- bus_business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable =

In [102]:
joined_df = res.join(reviewdf,res.bus_business_id==reviewdf.business_id,"left")

In [103]:
joined_df.count()

792133

In [108]:
joined_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- bus_business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable =

In [109]:
joined_df = joined_df.drop('business_id')

In [110]:
joined_df = joined_df.withColumnRenamed("bus_business_id","business_id")

In [111]:
joined_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = fal

In [112]:
joined_df.coalesce(1).write.json("data_joined_rs")

In [61]:
joined_df.coalesce(1).write.option("header",True).csv("data_joined_rs_csv")

In [65]:
joined_df_compact = spark.read.json('/home/paritosh/Downloads/business_joined.json')

In [66]:
joined_df_compact.count()

17923933

In [67]:
joined_df_compact.printSchema()

root
 |-- _corrupt_record: string (nullable = true)



In [105]:
joined_df.groupBy('stars').count().show()

23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:25:22 WARN RowBasedKeyValueBatch: Calling spill() on

+-----+------+
|stars| count|
+-----+------+
|    1| 99168|
|    3| 82636|
|    5|375217|
|    4|171651|
|    2| 63461|
+-----+------+



In [107]:
joined_df.groupBy('restaurant_stars').count().show()

23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.
23/03/05 02:27:18 WARN RowBasedKeyValueBatch: Calling spill() on RowBasedKeyValueBatch. Will not spill but return 0.


+----------------+------+
|restaurant_stars| count|
+----------------+------+
|             3.5|173934|
|             4.5|185417|
|             2.5| 32151|
|             1.0|   343|
|             4.0|305437|
|             3.0| 63525|
|             2.0| 11894|
|             1.5|  5475|
|             5.0| 13957|
+----------------+------+



In [36]:
spark.sql("select * from restaurants").count()

52268

In [36]:
rescat = spark.sql("select *, case when categories like '%American (Traditional)%' then 'Traditional American' \
                      when categories like '%American (New)%' then 'New American' \
                      when categories like '%Italian%' then 'Italian' \
                      when categories like '%Mexican%' then 'Mexican' \
                      when categories like '%Indian%' then 'Indian' \
                      when categories like '%Chinese%' then 'Chinese' \
                      when categories like '%Thai%' then 'Thai' \
                      when categories like '%Middle Eastern%' then 'Middle Eastern' \
                      when categories like '%Korean%' then 'Korean' \
                      when categories like '%Mediterranean%' then 'Mediterranean' \
                      when categories like '%Vietnamese%' then 'Vietnamese' \
                      else 'remove' end as Categories2 from restaurants")

In [37]:
rescat.count()

52268

In [38]:
rescat.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Al

In [39]:
rescat.select("Name","Categories","Categories2").show(5)

+--------------------+--------------------+--------------------+
|                Name|          Categories|         Categories2|
+--------------------+--------------------+--------------------+
|  St Honore Pastries|Restaurants, Food...|              remove|
|      Sonic Drive-In|Burgers, Fast Foo...|              remove|
|Tsevi's Pub And G...|Pubs, Restaurants...|Traditional American|
|      Sonic Drive-In|Ice Cream & Froze...|              remove|
|Vietnamese Food T...|Vietnamese, Food,...|          Vietnamese|
+--------------------+--------------------+--------------------+
only showing top 5 rows



In [40]:
rescat2 = rescat.drop("Categories","is_restaurants",)

In [41]:
rescat2 = rescat2.withColumnRenamed("Categories2","Categories")

In [42]:
rescat2.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Alcohol: string (nullable = false)
 |-- Driv

In [91]:
rescat2.select("Name","Categories").show(5)

+--------------------+--------------------+
|                Name|          Categories|
+--------------------+--------------------+
|  St Honore Pastries|              remove|
|      Sonic Drive-In|              remove|
|Tsevi's Pub And G...|Traditional American|
|      Sonic Drive-In|              remove|
|Vietnamese Food T...|          Vietnamese|
+--------------------+--------------------+
only showing top 5 rows



In [44]:
filrescat = rescat2.filter(rescat2.Categories!="remove")

In [45]:
filrescat.select("Name","Categories").count()

27330

In [46]:
filrescat.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Alcohol: string (nullable = false)
 |-- Driv

In [94]:
filrescat.coalesce(1).write.option("header",True).csv("/home/dbda/Project/dataset/Concentrated_Restaurants")

In [97]:
filrescat.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongo_ip + "business_restaurants").save()

## Reading the Review DataSet and joining the Restaurants with the Reviews ##

In [8]:
reviewdf = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "yelp_review").load()

In [9]:
reviewdf.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- cool: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)
 |-- user_id: string (nullable = true)



In [10]:
reviewdf.show(5)

+--------------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|                 _id|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|{63fc98d96125f1c4...|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|    3|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|{63fc98d96125f1c4...|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|    5|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|{63fc98d96125f1c4...|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|    3|Family diner. Had...|     0|8g_iMtfSiwikVnbP2...|
|{63fc98d96125f1c4...|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...

In [11]:
reviewdf_drop = reviewdf.drop("user_id","_id")
reviewdf_drop.show()

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|    3|If you decide to ...|     0|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|    5|I've taken a lot ...|     1|
|YjUWPpI6HXG530lwP...|   0|2014-02-05 20:30:30|    0|saUsX_uimxRlCVr67...|    3|Family diner. Had...|     0|
|kxX2SOes4o-D3ZQBk...|   1|2015-01-04 00:01:03|    0|AqPFMleE6RsU23_au...|    5|Wow!  Yummy, diff...|     1|
|e4Vwtrqf-wpJfwesg...|   1|2017-01-14 20:54:15|    0|Sx8TMOWLNuJBWer-0...|    4|Cute interior and...|     1|
|04UD14gamNjLY0IDY...|   1|2015-09-23 23:10:31|    2|JrIxlS1TzJ-iCu79u...|    1|I am a long term ...|     1|
|gmjsEdUsKpj9Xxu6p.

In [12]:
reviewdf_drop.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: integer (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: integer (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: integer (nullable = true)



In [47]:
ren_filrescat = filrescat.withColumnRenamed("stars","restaurant_stars").withColumnRenamed("business_id","bus_business_id")

In [48]:
ren_filrescat.printSchema()

root
 |-- address: string (nullable = true)
 |-- bus_business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Alcohol: string (nullable = f

In [49]:
spark.sql("select city,state,count(*) as count from restaurants group by city,state order by count desc").show()

+----------------+-----+-----+
|            city|state|count|
+----------------+-----+-----+
|    Philadelphia|   PA| 5852|
|           Tampa|   FL| 2960|
|    Indianapolis|   IN| 2862|
|       Nashville|   TN| 2502|
|          Tucson|   AZ| 2466|
|     New Orleans|   LA| 2259|
|        Edmonton|   AB| 2166|
|     Saint Louis|   MO| 1790|
|            Reno|   NV| 1286|
|           Boise|   ID|  847|
|   Santa Barbara|   CA|  767|
|      Clearwater|   FL|  678|
|      Wilmington|   DE|  619|
|       St. Louis|   MO|  542|
|        Metairie|   LA|  522|
|Saint Petersburg|   FL|  491|
|        Franklin|   TN|  442|
|  St. Petersburg|   FL|  404|
|          Sparks|   NV|  334|
|         Brandon|   FL|  326|
+----------------+-----+-----+
only showing top 20 rows



In [50]:
ren_filrescat = ren_filrescat.filter(ren_filrescat.state=="FL")

In [51]:
joined_df = ren_filrescat.join(reviewdf_drop,ren_filrescat.bus_business_id==reviewdf_drop.business_id,"left")

In [52]:
joined_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- bus_business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Alcohol: string (nullable = f

In [53]:
joined_df.count()

497887

In [54]:
drop_joined_df = joined_df.drop("business_id")
drop_joined_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- bus_business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Alcohol: string (nullable = f

In [55]:
drop_joined_df = drop_joined_df.withColumnRenamed("bus_business_id","business_id")
drop_joined_df.printSchema()

root
 |-- address: string (nullable = true)
 |-- business_id: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- name: string (nullable = true)
 |-- postal_code: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- restaurant_stars: double (nullable = true)
 |-- state: string (nullable = true)
 |-- DogsAllowed: string (nullable = false)
 |-- RestaurantsTakeOut: string (nullable = false)
 |-- HappyHour: string (nullable = false)
 |-- OutdoorSeating: string (nullable = false)
 |-- GoodForKids: string (nullable = false)
 |-- HasTV: string (nullable = false)
 |-- RestaurantsReservations: string (nullable = false)
 |-- WheelchairAccessible: string (nullable = false)
 |-- RestaurantsPriceRange: string (nullable = false)
 |-- Wifi: string (nullable = false)
 |-- Noiselevel: string (nullable = false)
 |-- Alcohol: string (nullable = false

In [56]:
drop_joined_df.count()

497887

In [96]:
drop_joined_df.coalesce(1).write.option("header",True).csv("/home/dbda/project/spark/data_joined.csv")

In [57]:
drop_joined_df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongo_ip + "business_review_joined").save()

## Data preprocessing for recommender system

In [45]:
df_temp = df.drop('_id', 'attributes', 'address', 'hours', 'postal_code','latitude','longitude')

In [46]:
df_temp.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- city: string (nullable = true)
 |-- is_open: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- review_count: integer (nullable = true)
 |-- stars: double (nullable = true)
 |-- state: string (nullable = true)



In [47]:
df_temp.createOrReplaceTempView("yelp_business")

In [48]:
df_temp = spark.sql("SELECT * FROM yelp_business WHERE categories LIKE '%Restaurants%'")

In [49]:
df_temp.count()

52268

In [44]:
df_temp.show()

+--------------------+--------------------+------------+-------+-------------+--------------+--------------------+------------+-----+-----+
|         business_id|          categories|        city|is_open|     latitude|     longitude|                name|review_count|stars|state|
+--------------------+--------------------+------------+-------+-------------+--------------+--------------------+------------+-----+-----+
|MTSW4McQd7CbVtyjq...|Restaurants, Food...|Philadelphia|      1|   39.9555052|   -75.1555641|  St Honore Pastries|          80|  4.0|   PA|
|CF33F8-E6oudUQ46H...|Burgers, Fast Foo...|Ashland City|      1|    36.269593|    -87.058943|      Sonic Drive-In|           6|  2.0|   TN|
|k0hlBqXX-Bt0vf1op...|Pubs, Restaurants...|      Affton|      0|   38.5651648|   -90.3210868|Tsevi's Pub And G...|          19|  3.0|   MO|
|bBDDEgkFA1Otx9Lfe...|Ice Cream & Froze...|   Nashville|      1|   36.2081024|   -86.7681696|      Sonic Drive-In|          10|  1.5|   TN|
|eEOYSgkmpB90uNA7l..

In [50]:
df_temp.createOrReplaceTempView("new")

In [54]:
df_temp = spark.sql("SELECT * FROM new WHERE state like '%FL%'")

In [55]:
df_temp.count()

8731

In [56]:
df_business_temp = df_temp

In [57]:
df_review_temp = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", mongo_ip + "yelp_review").load()

In [58]:
df_review_temp = df_review_temp.drop('_id', 'cool', 'date', 'funny', 'useful', 'user_id')

In [59]:
df_review_temp = df_review_temp.withColumnRenamed('business_id', 'business_id_rev')

In [60]:
df_review_temp.printSchema()

root
 |-- business_id_rev: string (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: integer (nullable = true)
 |-- text: string (nullable = true)



In [63]:
joined_df = df_business_temp.join(df_review_temp, df_business_temp.business_id==df_review_temp.business_id_rev, "left")

In [64]:
joined_df.count()

792133

In [65]:
joined_df.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongo_ip + "recommender_review_joined").save()

In [66]:
df_temp.write.format("com.mongodb.spark.sql.DefaultSource").mode("append").option("uri", mongo_ip + "recommender_business").save()